In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import boxcox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

from sdm.utils.download_fred_data import FredFxTimeSeries

from matplotlib.figure import Figure  # for typehints

from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.datasets import co2

# plt.style.use("dark_background")

In [34]:
# reloading our cached datasets

gbp = FredFxTimeSeries(name="gbp", series_id="DEXUSUK")
aud = FredFxTimeSeries(name="aud", series_id="DEXUSAL")
cad = FredFxTimeSeries(name="cad", series_id="DEXCAUS")
rmb = FredFxTimeSeries(name="rmb", series_id="DEXCHUS")
jpy = FredFxTimeSeries(name="jpy", series_id="DEXJPUS")
nzd = FredFxTimeSeries(name="nzd", series_id="DEXUSNZ")
spd = FredFxTimeSeries(name="spd", series_id="DEXSIUS")
chz = FredFxTimeSeries(name="chz", series_id="DEXSZUS")

datasets = [
    gbp,
    aud,
    cad,
    rmb,
    jpy,
    nzd,
    spd,
    chz
]

In [35]:
# adapted from Atwan 2022 pg.310
def stationarity_testing(out: tuple, test: str = "adf") -> pd.Series:
    p_value = out[1]
    score = out[0]
    lags = out[2]
    dec = "Non-Stationary"
    if test == "adf":
        crit = out[4]
        if p_value < 0.05:
            dec = "Stationary"
    elif test == "kpss":
        crit = out[3]
        if p_value >= 0.05:
            dec = "Stationary"
    else:
        raise ValueError("Available tests are: Augmented Dickey-Fuller ('adf'), or Kwiatkowski-Phillips-Shin ('kpss').")
    results = {
        "Test Statistic": score,
        "p-value": p_value,
        "Lags": lags,
        "Decision": dec
    }
    
    for k, v in crit.items():
        results[f"Citical Value {k}"] = v
        
    return pd.Series(results, name=test)

In [36]:
# stationarity testing for AUD

adf_out = adfuller(aud.data.loc["1990":"2016"]["OT"])
kpss_out = kpss(aud.data.loc["1990":"2016"]["OT"])

pd.concat([stationarity_testing(adf_out, "adf"), stationarity_testing(kpss_out, "kpss")], axis=1)

,adf,kpss
Test Statistic,0.108455,0.726226
p-value,0.966634,0.011161
Lags,4,3
Decision,Non-Stationary,Non-Stationary
Citical Value 1%,-3.769733,0.739
Citical Value 5%,-3.005426,0.463
Citical Value 10%,-2.642501,0.347
Citical Value 2.5%,NaN,0.574


In [69]:
# null for ljang-box is that there is no correlation between lags
# let's iterate over a stride checking for autocorrelation among lags
# across the entire time series then describe results

# aud.data.index = pd.to_datetime(aud.data["date"])

# del aud.data["date"]

first_order = aud.data.loc["1971-01-04":"2023-03-17"]["OT"].diff().dropna()

stride = 180
lags = 15
n = stride

start = pd.DataFrame(acorr_ljungbox(first_order.iloc[:stride], return_df=True, lags=lags)["lb_pvalue"])

while n + stride <= len(first_order):
    stationarity_test = first_order.iloc[n:n+stride]
    new = pd.DataFrame(acorr_ljungbox(stationarity_test, lags=lags, return_df=True)["lb_pvalue"])
    start[n] = new
    n += stride

s = start.std(axis=1)
mi = start.min(axis=1)
ma = start.max(axis=1)
me = start.mean(axis=1)

results = pd.DataFrame([s, mi, ma, me], ["std", "min", "max", "mean"])
results.T.to_latex()

/var/folders/40/ph9mx8691dq3w4pqhnfclk900000gn/T/ipykernel_14709/1369307230.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.T.to_latex()


'\\begin{tabular}{lrrrr}\n\\toprule\n{} &       std &           min &       max &      mean \\\\\n\\midrule\n1  &  0.318150 &  1.145070e-06 &  0.999961 &  0.414471 \\\\\n2  &  0.335637 &  4.677177e-07 &  0.991820 &  0.438377 \\\\\n3  &  0.335523 &  2.041557e-06 &  0.998017 &  0.422575 \\\\\n4  &  0.330418 &  1.705735e-07 &  0.986878 &  0.410366 \\\\\n5  &  0.329536 &  3.031203e-07 &  0.996757 &  0.410376 \\\\\n6  &  0.340637 &  5.668141e-07 &  0.998417 &  0.433262 \\\\\n7  &  0.320835 &  1.032062e-06 &  0.999549 &  0.419724 \\\\\n8  &  0.314956 &  2.088976e-06 &  0.999471 &  0.413903 \\\\\n9  &  0.311847 &  2.656233e-06 &  0.999835 &  0.420806 \\\\\n10 &  0.311809 &  5.710692e-06 &  0.999954 &  0.432333 \\\\\n11 &  0.311486 &  1.196236e-05 &  0.999963 &  0.437671 \\\\\n12 &  0.318278 &  2.463210e-05 &  0.999990 &  0.445910 \\\\\n13 &  0.325162 &  1.759260e-05 &  0.999997 &  0.450306 \\\\\n14 &  0.333881 &  8.983123e-06 &  0.999964 &  0.451544 \\\\\n15 &  0.327784 &  8.891652e-06 &  0.9

In [59]:
aud.data

,OT
date,
1971-01-04,1.1127
1971-01-05,1.1132
1971-01-06,1.1140
1971-01-07,1.1138
1971-01-08,1.1124
...,...
2023-03-13,0.6686
2023-03-14,0.6662
2023-03-15,0.6611
